In [11]:
import aug_util as aug
import wv_util as wv
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
import numpy as np
import csv
import tqdm
%matplotlib inline


In [12]:
#Loading our labels
coords1, chips1, classes1 = wv.get_labels('/data/zjc4/xView_train.geojson')

100%|██████████| 601937/601937 [00:03<00:00, 151894.72it/s]


In [7]:
np.unique(chips1).shape

(847,)

In [8]:
# Grouped classes [1,2,3,4,5]
grouped_classes = [[11,12],[13],[17,18,20,21],\
       [19,23,24,25,28,29,60,61,65,26],[41,42,50,40,44,45,47,49]]
def transpose_class_counts(chips,classes):
    chip_names = np.unique(chips) 
    ### Uncomment this to run over entire train dataset ###
    # chip_names = np.unique(chips) 
    results = np.zeros((len(chip_names),5))
    chip_strs = []
    for c_idx, c in tqdm.tqdm(enumerate(chip_names)):
        chip_strs.append(c)
        classes_chip = classes[chips==c]
        idx_filter = np.isin(classes_chip,grouped_classes[0])
        # initialize to all false
        for i,gc in (enumerate(grouped_classes)):
            is_in_idxs = np.isin(classes_chip,gc)
            classes_chip[is_in_idxs] = i
            idx_filter = np.logical_or(idx_filter,is_in_idxs)
        classes_chip = classes_chip[idx_filter]
        labels, counts = np.unique(classes_chip,return_counts=True)
        for label_idx,label in enumerate(labels):
            results[int(c_idx),int(label)] = counts[label_idx]
            pass
    chip_strs_col = np.array(chip_strs).reshape(-1,1)
    return (np.hstack((chip_strs_col,results)))

# make the table where filename is rowwise, columns are class
results = transpose_class_counts(chips1,classes1)

847it [00:29, 28.92it/s]


In [14]:
columns = \
    ['fname',
    'Small Aircraft',
    'Large Aircraft',
    'Small Vehicle',
    'Bus/Truck',
    'Boat']

df = pd.DataFrame(results)
df.columns = columns
#df.head(10)

In [15]:
df.to_csv('all_tifs.csv')

In [16]:
df

,fname,Small Aircraft,Large Aircraft,Small Vehicle,Bus/Truck,Boat
0,10.tif,0.0,0.0,2.0,1.0,0.0
1,100.tif,0.0,0.0,62.0,7.0,0.0
2,102.tif,0.0,0.0,346.0,24.0,0.0
3,1036.tif,0.0,0.0,31.0,36.0,69.0
4,1037.tif,0.0,0.0,276.0,146.0,0.0
...,...,...,...,...,...,...
842,97.tif,0.0,0.0,934.0,61.0,0.0
843,983.tif,1.0,2.0,0.0,1.0,0.0
844,986.tif,0.0,0.0,7.0,3.0,0.0
845,99.tif,0.0,0.0,851.0,81.0,0.0
